In [ ]:
# import os
# os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install youtokentome

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.tensorboard import SummaryWriter

import youtokentome as yttm
import numpy as np

import random
import math
import time
import pickle
from tqdm.auto import tqdm

In [ ]:
# %env CUDA_LAUNCH_BLOCKING=1

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# torch.use_deterministic_algorithms(True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [ ]:
# token_model = yttm.BPE(model='try_poems_embed_yttm.model', n_threads=-1)
token_model = yttm.BPE(model='/content/drive/MyDrive/Colab Notebooks/NLP_poems/voc10k_poems_embed_yttm.model', n_threads=-1)

In [ ]:
# with open('data/try_poems_preproc.pickle', 'rb', ) as file:
with open('/content/drive/MyDrive/Colab Notebooks/NLP_poems/data/voc10k_poems_preproc.pickle', 'rb', ) as file:
    preproc_data = pickle.load(file)

In [ ]:
token_data = token_model.encode(preproc_data, output_type=yttm.OutputType.ID, bos=True, eos=True)

In [ ]:
# print(token_model.vocab())
# print(len(token_model.vocab()))
# print(preproc_data[2])
# print(token_data[2])
# print(len(preproc_data[1].split(' ')))
# print(len(token_data[1]))

In [ ]:
class newDataset(Dataset):
    def __init__(self, data_list_of_list):
        self.data = data_list_of_list

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

    def cut_length(self, limit):
#         middle_len = sum([len(text) for text in self.data])/len(self.data)
#         limit = int(middle_len)*2
        self.data = [text if len(text)<limit else text[:limit] for text in self.data]

    def distribute_data(self, share_tr, share_val):  # examle = (0.8, 0.1)
        rand_i = np.random.permutation(len(self.data))
        n1 = int(len(self.data) * share_tr)
        n2 = int(len(self.data) * share_val)
        # return self.data[rand_i[0: n1]], self.data[rand_i[n1: n1 + n2]], self.data[rand_i[n1 + n2:]]
        return [self.data[i] for i in rand_i[0: n1]], \
               [self.data[i] for i in rand_i[n1: n1 + n2]], \
               [self.data[i] for i in rand_i[n1 + n2:]]
    
def padding(batch):
    pad_id = token_model.subword_to_id('<PAD>')
    
    batch = [torch.tensor(x) for x in batch]
    batch = pad_sequence(batch, batch_first=False, padding_value=pad_id)  # batch_first=True -> [batch size, len]
    return batch
    # return batch.to(device)

def sort_padding(batch):
    pad_id = token_model.subword_to_id('<PAD>')
    
    batch.sort(key = lambda x : -len(x)) # first longest

    batch_lens = [torch.tensor(len(x)) for x in batch]
    batch = [torch.tensor(x) for x in batch]
    batch = pad_sequence(batch, batch_first=False, padding_value=pad_id)  # batch_first=True -> [batch size, len]
    return batch, torch.tensor(batch_lens)

In [ ]:
token_dataset = newDataset(token_data)
token_dataset.cut_length(40)
train_data, valid_data, test_data = token_dataset.distribute_data(0.8, 0.1)
train_loader, valid_loader, test_loader = \
    DataLoader(train_data, batch_size=128, shuffle=True, num_workers=0, pin_memory=True, collate_fn=sort_padding), \
    DataLoader(valid_data, batch_size=128, shuffle=True, num_workers=0, pin_memory=True, collate_fn=sort_padding), \
    DataLoader(test_data, batch_size=64, shuffle=False, num_workers=0, pin_memory=True, collate_fn=sort_padding)

# with open('data/try_test_data_token.pickle', 'wb', ) as file:
# with open('/content/drive/MyDrive/Colab Notebooks/NLP_poems/data/voc10klen100_test_data_token.pickle', 'wb', ) as file:
#     pickle.dump(test_data, file)

In [ ]:
# print(len(train_data))
# print(train_loader.batch_size)
# print(len(train_loader))
# for b in train_loader:
#   print(type(b[0]))
#   print(type(b[1]))
#   break

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers) # dropout = dropout
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_nopad_len):      
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        #embedded = [src len, batch size, emb dim]
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_nopad_len.to('cpu'))
                
        packed_outputs, hidden = self.rnn(packed_embedded)
        #packed_outputs is a packed sequence containing all hidden states
        #hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
        #outputs is now a non-packed sequence
        
        #outputs = [src len, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        #hidden - it will be first hid states in decoder
        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()
        
        self.attn = nn.Linear(hid_dim * 2, hid_dim)
        self.v = nn.Linear(hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, hid dim]
        #encoder_outputs = [src len, batch size, hid dim]
        #mask = [batch size, src len]  
        # example: sent=['hello', <pad>, <pad>], mask=[1,0,0]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        #hidden = [batch size, src len, hid dim]
        #encoder_outputs = [batch size, src len, hid dim]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        #energy = [batch size, src len, hid dim]

        attention = self.v(energy).squeeze(2)
        attention = attention.masked_fill(mask == 0, -1e10)
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout, attention):
        super().__init__()
        
        self.output_dim = output_dim  # the size of the vocabulary for the output/target.
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim) # 1 - size of dict emb, 2 - size of emb vec
        
        self.rnn = nn.GRU(hid_dim + emb_dim, hid_dim, n_layers) # dropout = dropout
        
        self.fc_out = nn.Linear(hid_dim * 2 + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, mask):

        #input = [batch size]
        #hidden = [n layers, batch size, hid dim]
        #encoder_outputs = [src len, batch size, hid dim]
        
        input = input.unsqueeze(0) 
        #input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        #embedded = [1, batch size, emb dim]
                
        last_hidden = hidden[-1]
        #hidden = [batch size, hid dim]        
        a = self.attention(last_hidden, encoder_outputs, mask)    
        #a = [batch size, src len]
        a = a.unsqueeze(1)
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        #encoder_outputs = [batch size, src len, hid dim]
        
        weighted = torch.bmm(a, encoder_outputs)    
        #weighted = [batch size, 1, hid dim]
        weighted = weighted.permute(1, 0, 2)
        #weighted = [1, batch size, enc hid dim]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        #rnn_input = [1, batch size, hid dim + emb dim]
        
        output, hidden = self.rnn(rnn_input, hidden)
        #output = [1, batch size, hid dim], seq len will always be 1 in the decoder
        #hidden = [n layers, batch size, hid dim]
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device

    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_nopad_len, trg, teacher_forcing_ratio = 0.2):
        
        #src = [src len, batch size]
        #src_nopad_len = [batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio=0.75 -> use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence
        #hidden is the final hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_nopad_len)
        
        #first input to the decoder is the <bos> tokens
        input = trg[0,:]  
        mask = self.create_mask(src)
        #mask = [batch size, src len]
        
        for t in range(1, trg_len):
            
            #insert input token emb-g, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs, mask)
            
            outputs[t] = output
            
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token, else use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [ ]:
INPUT_DIM = token_model.vocab_size() 
OUTPUT_DIM = INPUT_DIM
ENC_EMB_DIM = 128 #128
DEC_EMB_DIM = 128 #128
HID_DIM = 256 #256
N_LAYERS = 2 
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2
PAD_ID = token_model.subword_to_id('<PAD>')

attn = Attention(HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, PAD_ID, device).to(device)
# writer = SummaryWriter(log_dir='runs/Adam0001_Cycle005_dr02_e20_voc10k')
writer = SummaryWriter(log_dir='/content/drive/MyDrive/Colab Notebooks/NLP_poems/runs/AdamW01_C005_drEm2_voc10k_tf2_len40_AttPad')

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,680,592 trainable parameters


In [ ]:
N_EPOCHS = 20
CLIP = 1

# optimizer = optim.Adam(model.parameters(), lr=0.0001)
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.01) 

# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=60, gamma=0.1)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr = 0.005, total_steps = len(train_loader)*N_EPOCHS, cycle_momentum=True
)

criterion = nn.CrossEntropyLoss(ignore_index = PAD_ID)

In [ ]:
def train(model, loader, optimizer, scheduler, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(tqdm(loader)):
        batch_src = batch[0].to(device, non_blocking=True)
        batch_src_nopad_len = batch[1]
        src, src_nopad_len = batch_src, src_nopad_len
        trg = batch_src
        
        optimizer.zero_grad()
        
        output = model(src, src_nopad_len, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        loss.backward()
        
#         torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        scheduler.step() # for Cycle
        
        epoch_loss += loss.item()
        
#     scheduler.step() # for SGD
        
    return epoch_loss / len(loader)

In [ ]:
def evaluate(model, loader, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(tqdm(loader)):
            batch_src = batch[0].to(device, non_blocking=True)
            src, src_nopad_len = batch_src, batch[1]
            trg = batch_src

            output = model(src, src_nopad_len, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(loader)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=None, abbreviated=False)  # give info about the alloaction of memory

# torch.multiprocessing.set_start_method('spawn')  # for num_workers in Dataloader
# torch.cuda.device_count()

In [ ]:
# checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/NLP_poems/b7e20n2_10k_checkpoint.pt')
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
# start_epoch = checkpoint['epoch']
# bv_loss = checkpoint['loss']

In [ ]:
bv_loss = float('inf')
best_valid_loss = bv_loss 

start_epoch = 0
for epoch in range(start_epoch, start_epoch + N_EPOCHS):
      
    start_time = time.time()
        
    train_loss = train(model, train_loader, optimizer, scheduler, criterion, CLIP)
    valid_loss = evaluate(model, valid_loader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Loss/valid", valid_loss, epoch)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': best_valid_loss
            # }, 'AdamW01_Cycle005_dr02_e20_checkp_best.pt')
            }, '/content/drive/MyDrive/Colab Notebooks/NLP_poems/AdamW01_C005_drEm2_voc10k_tf2_len40_AttPad_checkp_best.pt')

    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'loss': valid_loss
        # }, 'AdamW01_Cycle005_dr02_e20_checkp_last.pt')
            }, '/content/drive/MyDrive/Colab Notebooks/NLP_poems/AdamW01_C005_drEm2_voc10k_tf2_len40_AttPad_checkp_last.pt')
        
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    
writer.flush()
writer.close()

  0%|          | 0/112 [00:00<?, ?it/s]

ValueError: ignored

In [ ]:
writer.flush()
writer.close()

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir mylogdir